In [2]:
import numpy as np
import pandas as pd
import transformers
import torch
from transformers import BertTokenizer,BertModel
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer
from sentence_transformers import evaluation
from math import floor, log10
from sklearn.metrics import mean_squared_error

In [3]:
df=pd.read_csv("ds.csv")

In [4]:
df

,id,question,desired_answer,student_answer,score_me,score_other,score_avg
0,1.10,What is the role of a prototype program in pro...,To simulate the behaviour of portions of the d...,High risk problems are address in the prototyp...,4.0,3.00,3.500
1,1.10,What is the role of a prototype program in pro...,To simulate the behaviour of portions of the d...,To simulate portions of the desired final prod...,5.0,5.00,5.000
2,1.10,What is the role of a prototype program in pro...,To simulate the behaviour of portions of the d...,A prototype program simulates the behaviors of...,5.0,3.00,4.000
3,1.10,What is the role of a prototype program in pro...,To simulate the behaviour of portions of the d...,Defined in the Specification phase a prototype...,5.0,5.00,5.000
4,1.10,What is the role of a prototype program in pro...,To simulate the behaviour of portions of the d...,It is used to let the users have a first idea ...,3.0,3.00,3.000
...,...,...,...,...,...,...,...
2268,12.11,How many steps does it take to search a node i...,The height of the tree.,log n,4.5,5.00,4.750
2269,12.11,How many steps does it take to search a node i...,The height of the tree.,( n(n-1) ) / 2,0.0,3.00,1.500
2270,12.11,How many steps does it take to search a node i...,The height of the tree.,2n-1,0.0,4.75,2.375
2271,12.11,How many steps does it take to search a node i...,The height of the tree.,"it takes at most h steps, where h is the heigh...",5.0,5.00,5.000


In [5]:
bert_base = SentenceTransformer('bert-base-nli-mean-tokens')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [6]:
bert_base.named_parameters

<bound method Module.named_parameters of SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)>

In [7]:
student_answers_tk = df['student_answer'].to_list()
ref_answers_tk=df['desired_answer'].to_list()
enc_ref_ans=[]
enc_stud_ans=[]
# "[CLS] " + text + " [SEP]"

for i,j in zip(student_answers_tk,ref_answers_tk):
    enc_stud_ans.append(tokenizer.encode(i))
    enc_ref_ans.append(tokenizer.encode(j))
print(enc_ref_ans[0])
# CLS=101 SEP=102

[101, 2000, 26633, 1996, 9164, 1997, 8810, 1997, 1996, 9059, 4007, 4031, 1012, 102]


In [8]:
list(tokenizer.vocab.keys())[100:150]

['[UNK]',
 '[CLS]',
 '[SEP]',
 '[MASK]',
 '[unused99]',
 '[unused100]',
 '[unused101]',
 '[unused102]',
 '[unused103]',
 '[unused104]',
 '[unused105]',
 '[unused106]',
 '[unused107]',
 '[unused108]',
 '[unused109]',
 '[unused110]',
 '[unused111]',
 '[unused112]',
 '[unused113]',
 '[unused114]',
 '[unused115]',
 '[unused116]',
 '[unused117]',
 '[unused118]',
 '[unused119]',
 '[unused120]',
 '[unused121]',
 '[unused122]',
 '[unused123]',
 '[unused124]',
 '[unused125]',
 '[unused126]',
 '[unused127]',
 '[unused128]',
 '[unused129]',
 '[unused130]',
 '[unused131]',
 '[unused132]',
 '[unused133]',
 '[unused134]',
 '[unused135]',
 '[unused136]',
 '[unused137]',
 '[unused138]',
 '[unused139]',
 '[unused140]',
 '[unused141]',
 '[unused142]',
 '[unused143]',
 '[unused144]']

In [9]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
student_answers_tk = df['student_answer'].to_list()
ref_answers_tk=df['desired_answer'].to_list()
enc_ref_ans=[]
enc_stud_ans=[]
for i,j in zip(student_answers_tk,ref_answers_tk):
    enc_stud_ans.append(tokenizer.encode(i))
    enc_ref_ans.append(tokenizer.encode(j))

In [10]:
bert_base = SentenceTransformer('bert-base-nli-mean-tokens')

df = pd.read_csv('ds.csv')


student_answers = df['student_answer'].to_list()
ref_answers=df['desired_answer'].to_list()
scores_average=df['score_avg'].to_list()
pred_scores=[]
sim_arr=[]

for i,j in zip(ref_answers,student_answers):
    embed_array=[]
    embed_array.append(i)
    embed_array.append(j)
    embed_array_res=bert_base.encode(embed_array)

    res=cosine_similarity([embed_array_res[0]],embed_array_res[1:])[0][0]
    sim_arr.append(res)
    pred_scores.append(res*5)

In [11]:
pred_scores_rounded=[round(i* 2) / 2 for i in pred_scores]
print(pred_scores_rounded)

[3.5, 3.5, 4.0, 4.0, 3.5, 3.5, 2.0, 3.0, 2.0, 3.5, 4.5, 5.0, 3.0, 4.0, 3.5, 4.0, 5.0, 3.0, 3.0, 3.0, 5.0, 5.0, 4.0, 3.0, 5.0, 4.0, 2.5, 3.5, 2.5, 3.0, 3.0, 3.5, 3.5, 3.5, 3.5, 2.5, 3.0, 1.5, 3.0, 2.0, 3.0, 2.5, 2.0, 3.5, 2.5, 3.0, 2.5, 2.5, 2.5, 2.5, 3.0, 2.5, 2.5, 3.0, 3.0, 3.0, 2.5, 3.5, 1.5, 1.5, 2.5, 2.5, 2.5, 2.5, 3.0, 2.0, 3.0, 3.0, 2.0, 2.5, 2.5, 3.0, 2.5, 1.5, 2.5, 2.5, 2.0, 2.0, 2.5, 2.5, 2.5, 2.5, 2.5, 2.5, 2.5, 2.5, 1.5, 4.5, 4.5, 3.0, 3.0, 4.5, 4.0, 4.5, 3.5, 3.0, 4.0, 4.0, 4.5, 3.5, 4.5, 3.0, 4.5, 4.5, 4.5, 4.5, 3.0, 4.5, 4.0, 3.0, 4.5, 3.0, 3.5, 3.0, 4.5, 4.5, 3.0, 3.0, 4.5, 4.0, 3.5, 3.5, 3.0, 3.5, 2.5, 4.0, 4.0, 4.5, 3.5, 4.5, 3.5, 3.5, 3.5, 3.0, 4.0, 3.0, 4.5, 4.5, 4.0, 4.5, 4.5, 3.5, 3.5, 4.5, 3.5, 2.0, 3.0, 3.5, 4.0, 3.0, 2.0, 3.0, 3.5, 2.5, 4.0, 3.0, 2.5, 2.0, 3.5, 3.0, 3.0, 1.5, 4.0, 3.0, 4.0, 1.5, 2.5, 4.0, 3.5, 4.0, 3.5, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 2.0, 3.0, 3.0, 3.5, 2.5, 3.0, 3.5, 4.0, 3.5, 2.5, 3.0, 3.5, 3.0, 3.5, 3.5, 3.0, 3.5, 3.5, 3.0, 2.5, 3.0, 1.0,

In [12]:
mean_squared_error(scores_average,pred_scores)/5

0.3702795947969567